In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from mapk import *

np.random.seed(2018)

# 데이터를 불러온다.
trn = pd.read_csv('./dataset/train_ver2.csv')
tst = pd.read_csv('./dataset/test_ver2.csv')

## 데이터 전처리 ##

# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체한다.
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거한다.
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 제품 변수는 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list이다.
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

# (피쳐 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

# (피쳐 엔지니어링) lag-1 데이터를 생성한다.
# 코드 2-12와 유사한 코드 흐름이다.

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

###
### Baseline 모델 이후, 다양한 피쳐 엔지니어링을 여기에 추가한다.
###


## 모델 학습
# 학습을 위하여 데이터를 훈련, 테스트용으로 분리한다.
# 학습에는 2016-01-28 ~ 2016-04-28 데이터만 사용하고, 검증에는 2016-05-28 데이터를 사용한다.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y


# XGBoost 모델 parameter를 설정한다.
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 8,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

# XGBoost 모델을 전체 훈련 데이터로 재학습한다!
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
# XGBoost 모델 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

# 변수 중요도를 출력해본다. 예상하던 변수가 상위로 올라와 있는가?
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

# 캐글 제출을 위하여 테스트 데이터에 대한 예측 값을 구한다.
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

# 제출 파일을 생성한다.
submit_file = open('./model/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))



E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:145: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:146: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[0]	train-mlogloss:2.77959
[1]	train-mlogloss:2.48942
[2]	train-mlogloss:2.30662
[3]	train-mlogloss:2.16682
[4]	train-mlogloss:2.0523
[5]	train-mlogloss:1.95127
[6]	train-mlogloss:1.87573
[7]	train-mlogloss:1.80123
[8]	train-mlogloss:1.73604
[9]	train-mlogloss:1.68193
[10]	train-mlogloss:1.6308
[11]	train-mlogloss:1.58826
[12]	train-mlogloss:1.54855
[13]	train-mlogloss:1.5143
[14]	train-mlogloss:1.48184
[15]	train-mlogloss:1.45171
[16]	train-mlogloss:1.42467
[17]	train-mlogloss:1.40003
[18]	train-mlogloss:1.37783
[19]	train-mlogloss:1.35797
[20]	train-mlogloss:1.33823
[21]	train-mlogloss:1.32002
[22]	train-mlogloss:1.30312
[23]	train-mlogloss:1.28776
[24]	train-mlogloss:1.27389
[25]	train-mlogloss:1.26121
[26]	train-mlogloss:1.24913
[27]	train-mlogloss:1.23751
[28]	train-mlogloss:1.22743
[29]	train-mlogloss:1.21746
[30]	train-mlogloss:1.20883
[31]	train-mlogloss:1.20007
[32]	train-mlogloss:1.19204
[33]	train-mlogloss:1.18453
[34]	train-mlogloss:1.17718
[35]	train-mlogloss:1.17064
[36]	

[285]	train-mlogloss:0.953616
[286]	train-mlogloss:0.953174
[287]	train-mlogloss:0.952772
[288]	train-mlogloss:0.952379
[289]	train-mlogloss:0.951791
[290]	train-mlogloss:0.951335
[291]	train-mlogloss:0.950956
[292]	train-mlogloss:0.950557
[293]	train-mlogloss:0.950119
[294]	train-mlogloss:0.949707
[295]	train-mlogloss:0.949244
[296]	train-mlogloss:0.948932
[297]	train-mlogloss:0.948635
[298]	train-mlogloss:0.948275
[299]	train-mlogloss:0.947801
[300]	train-mlogloss:0.947403
[301]	train-mlogloss:0.947093
[302]	train-mlogloss:0.946727
[303]	train-mlogloss:0.946343
[304]	train-mlogloss:0.945998
[305]	train-mlogloss:0.945574
[306]	train-mlogloss:0.945274
[307]	train-mlogloss:0.944923
[308]	train-mlogloss:0.944536
[309]	train-mlogloss:0.944216
[310]	train-mlogloss:0.943957
[311]	train-mlogloss:0.943653
Feature importance:
('renta', 27043)
('age', 23947)
('antiguedad', 23138)
('age_prev', 16463)
('antiguedad_prev', 16266)
('fecha_alta_month', 15235)
('nomprov', 14467)
('fecha_alta_year', 11

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:160: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:163: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:164: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [7]:
# 제출 파일을 생성한다.
count = 0
submit_file = open('./model/xgb.baseline.2019-06-17', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
#     print('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
#     count += 1
#     if count > 100:
#         break
        
submit_file.close()

In [8]:
len(ncodpers_tst)

929615